# Dataset Index Generation
Generates indices for train, validation and test sets

In [1]:
import h5py
from progressbar import *
import re
import numpy as np

## Options

In [2]:
# define which files of events are included in test / validation / training split
# the same split is done for each particle type
test_files_start = 0
test_files_stop = test_files_start+400 # first 400 files are for test set
val_files_start = test_files_stop
val_files_stop = val_files_start+100 # next 100 files are for validation set
train_files_start = val_files_stop
train_files_stop = None # all remaining files are for training set
# train_files_stop = 500 # next 500 files are for training set

# define which particle labels to include
labels = (0, 1, 2, 3)

## Load dataset

In [3]:
#data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/tts_egamma/IWCD_mPMT_Short_eg_1p5ns.h5"
data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/IWCD_mPMT_Short_emgp0_E0to1000MeV_digihits.h5"
f = h5py.File(data_path, "r")
event_labels = np.array(f['labels'])
root_files = np.array(f['root_files']).astype(str)

## Find the files of each label and indices of each file

In [4]:
def atoi(text):
    return int(text) if text.isdigit() else text
# Sort by only the basename of the file, with natural sorting of numbers in the filename
def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text.split('/')[-1]) ]

In [5]:
files_in_labels = {l: sorted(set(root_files[event_labels==l]), key=natural_keys) for l in labels}
idxs_in_files = {f: range(i, i+c) for f,i,c in zip(*np.unique(root_files, return_index=True, return_counts=True))}

In [6]:
for l, f in files_in_labels.items():
    print("label", l,"has", len(f),"files and ", sum([len(idxs_in_files[i]) for i in f]), "indices")

label 0 has 3000 files and  8868592 indices
label 1 has 3000 files and  8833531 indices
label 2 has 1000 files and  2911072 indices
label 3 has 1000 files and  2868354 indices


## Create the splits

In [7]:
split_files = {"test_idxs":  [f for l in labels for f in files_in_labels[l][test_files_start:test_files_stop]],
               "val_idxs":   [f for l in labels for f in files_in_labels[l][val_files_start:val_files_stop]],
               "train_idxs": [f for l in labels for f in files_in_labels[l][train_files_start:train_files_stop]]}
split_idxs = {k: [i for f in v for i in idxs_in_files[f]] for k, v in split_files.items()}

In [8]:
for s in split_files.keys():
    print(s,"has", len(split_files[s]),"files and", len(split_idxs[s]),"indices")

test_idxs has 1600 files and 4671520 indices
val_idxs has 400 files and 1168263 indices
train_idxs has 6000 files and 17641766 indices


In [9]:
# Verify that all events are uniquely accounted for
all_indices = np.concatenate(list(split_idxs.values()))
print(len(event_labels))
print(len(all_indices))
print(len(set(all_indices)))

23481549
23481549
23481549


## Save file

In [10]:
#np.savez('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/tts_egamma/IWCD_mPMT_Short_eg_1p5ns_idxs.npz', **split_idxs)
#np.savez('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/tts_egamma/IWCD_mPMT_Short_eg_2p5ns_idxs.npz', **split_idxs)
#np.savez('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/index_lists/2class_e_gamma/IWCD_mPMT_Short_2_class_eg_9M_all_test.npz', **split_idxs)
np.savez('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/index_lists/4class_e_mu_gamma_pi0/IWCD_mPMT_Short_4_class_3M_emgp0_fixed_idxs.npz', **split_idxs)